In [ ]:
 !bw2 status

### Make an entity

This creates a keypair you can use to interact with BOSSWAVE resources

In [ ]:
!bw2 mke -o myentity.ent

We want to interact with resources on `$NAMESPACE`. We first verify if we can do so by running the following command:

In [ ]:
!echo $NAMESPACE
!bw2 bc -t myentity.ent -u $NAMESPACE/*

Now, we need to ask the owner of the namespace for permission to operate on their namespace. 

In [ ]:
!./getaccess myentity.ent

Re-run this next line until "Registry" is marked **valid**

In [ ]:
!bw2 i <DOT HASH HERE>

Now, we can create a Python client with our entity and connect to our local agent

In [ ]:
from client import get_client
c = get_client(entity='myentity.ent')
import os
namespace = os.environ["NAMESPACE"]

In [ ]:
def mycallback(msg):
    print msg.uri, msg.payload
c.subscribe(namespace+"/*", mycallback)

In [ ]:
from widgets import Widgets
w = Widgets("scratch.ns/switch")
w.display()